In [16]:
import datacube
dc = datacube.Datacube(app='Sentinel_2')

lat, lon = 9.6, 28.43
buffer = 0.005
query = {
    'time': ('2020-10-1', '2020-10-10'),
    'x': (lon - buffer, lon + buffer),
    'y': (lat + buffer, lat - buffer),
    'output_crs': 'epsg:32635',
    'resolution': (-10, 10)
}

In [17]:
#from utils.deafrica_datahandling import load_ard
bands = ['blue', 'green', 'red', 'SCL']
ds = dc.load(poduct='s2_l2a',
             measurements=bands,
             dask_chunks={},
             **query)
ds

ValueError: Must specify a product or supply datasets

In [11]:
from utils.deafrica_plotting import rgb
rgb(ds, col='time', size=12)

ValueError: Can't determine shape from dimension names: 

In [4]:
from xarray import DataArray
import numpy as np
from typing import Union

def create_s2_mosaic(dataset_in: DataArray, no_data: Union[int, float] = 0) -> (DataArray, DataArray):
    """ Creates a most-recent-to-oldest mosaic of the input dataset.

    dataset_in: A dataset retrieved from the Data Cube; should contain:
        coordinates: time, latitude, longitude
        variables: variables to be mosaicked (e.g. red, green, and blue bands)
    no_data: No data value, default 0 """
    dataset_in = dataset_in.copy(deep=True)
    dataset_keys = list(dataset_in.data_vars)
    dataset_out = dataset_in.isel(time=-1).drop('time').copy(deep=True)
    dataset_out_array = dataset_out.values
    cols, rows = dataset_out.sizes['x'], dataset_out.sizes['y']
    # utilities.clear_attrs(dataset_out)  # TODO: import
    # dataset_out_vals = {key: dataset_out[key].values for key in dataset_keys}

    latest_time = ds.time[-1].values.astype('datetime64[D]').astype('uint16')
    recentness_matrix = np.empty((rows, cols), dtype=np.uint16)
    recentness_matrix[:] = latest_time

    for index in range(len(dataset_in.time) - 2, -1, -1):
        dataset_slice = dataset_in.isel(time=index).drop('time')
        dataset_slice_time = dataset_in.time[index].values.astype('datetime64[D]').astype('uint16')
        recentness_matrix[dataset_out_array == 0] = dataset_slice_time
        dataset_out_array[dataset_out_array == 0] = dataset_slice.values[dataset_out_array == 0]

    """
    for index in range(len(dataset_in.time) - 2, -1, -1):
        dataset_slice = dataset_in.isel(time=index).drop('time')
        dataset_slice_time = dataset_in.time[index].values.astype('datetime64[D]').astype('uint16')
        for key in dataset_keys:
            dataset_slice_values = None
            for i in range(rows):
                for j in range(cols):
                    val = dataset_out_vals[key][i][j]
                    if np.isnan(val) or val == no_data:
                        if dataset_slice_values is None:
                            dataset_slice_values = dataset_slice[key].values
                        dataset_out_vals[key][i][j] = dataset_slice_values[i][j]
                        recentness_matrix[i][j] = dataset_slice_time
    """

    dataset_out.values = dataset_out_array
    # for key in list(dataset_out.data_vars):
    #     dataset_out[key].values = dataset_out_vals[key]
    recentness_data = dataset_out.copy(deep=True)
    recentness_data.values = recentness_matrix
    # TODO: return a DataSet instead of (DataArray, DataArray)
    return dataset_out, recentness_data

In [26]:
r = ds.red
latest_r = r.isel(time=-1).drop('time')
r_arr = np.nan_to_num(latest_r.values, copy=False)
next_r = r.isel(time=-2).drop('time')
r_arr[r_arr==0] = next_r.values[r_arr==0]